In [11]:
from data_storage import create_connection
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix


In [12]:
connection = create_connection("../database/crypto_billionairs.db")

In [13]:
def apply_random_forest(X_train, X_test, y_train, y_test, table, db_connection):
    
    clf = RandomForestClassifier(criterion="entropy", min_samples_split= 0.01, min_samples_leaf= 0.005, max_depth=10, class_weight="balanced_subsample")
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    
    df.to_sql(f"{table}_random_forest_resampled", db_connection, if_exists="replace")
    return "random forest", f1score, f1score_macro, recall, precision

In [14]:
def apply_knn(X_train, X_test, y_train, y_test, table, db_connection):
    
    neigh = KNeighborsClassifier(weights="uniform", n_neighbors=5, algorithm="ball_tree")
    neigh.fit(X_train, y_train)
    y_pred = neigh.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_knn_resampled", db_connection, if_exists="replace")
    
    return "k-nearest neighbour", f1score, f1score_macro, recall, precision

In [15]:
def apply_support_vector_machine(X_train, X_test, y_train, y_test, table, db_connection):
    
    svc = SVC(kernel="poly", degree=4, C=1)
    svc.fit(X_train, y_train)
    
    y_pred = svc.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_svc_resampled", db_connection, if_exists="replace")
    
    return "support vector classifier", f1score, f1score_macro, precision, recall

In [16]:
def apply_mlp(X_train, X_test, y_train, y_test, table, db_connection):
    
    mlp = MLPClassifier(hidden_layer_sizes=(10, 10), activation="tanh", solver="lbfgs", learning_rate="constant", learning_rate_init=2e-5, tol=1e-5)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_mlp_resampled", db_connection, if_exists="replace")
    
    return "mlp classifier", f1score, f1score_macro, precision, recall 

In [17]:
def apply_logistic_regression(X_train, X_test, y_train, y_test, table, db_connection):
    
    lg = LogisticRegression(solver="liblinear", penalty="l1", C=1)
    lg.fit(X_train, y_train)
    y_pred = lg.predict(X_test)
    f1score = f1_score(y_test, y_pred, average="weighted")
    f1score_macro = f1_score(y_test, y_pred, average="macro")
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro")
    
    y_pred = y_pred.tolist()
    df = pd.concat([X_test])
    df["buy_short_indicator"] = y_pred
    df['close_buy_short_indicator'] = df["buy_short_indicator"].shift(1).fillna(0.0)
    df.to_sql(f"{table}_logistic_regression_resampled", db_connection, if_exists="replace")
    
    return "logistic regression", f1score, f1score_macro, recall, precision
    

In [18]:
def apply_ml_algorithms(db_connection):
    
    df_ml = pd.DataFrame(columns = range(6))
    df_ml.columns = ["table_name", "model", "f1-score weighted", "f1-score macro", "recall macro", "precision macro"]
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and 'equity_curve' not in name and '_pooling' not in name]
    ros = RandomOverSampler(random_state=0)
    print(filtered_table_names)
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        
        y = df_temp["buy_indicator"] + df_temp["short_indicator"]
        y = y.fillna(0)
        y = y.astype(str)
        
        X = df_temp.drop(["return", "buy_indicator", "short_indicator","close_buy_indicator", "close_short_indicator", "time", "index", "market_cap"], axis=1)
        
        X_train = X.iloc[:-365]
        X_test = X.iloc[-365:]
        
        y_train = y.iloc[:-365]
        y_test = y.iloc[-365:]
        
        X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
        
        string, score, f1score_macro, recall, precision  = apply_random_forest(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_knn(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_support_vector_machine(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_mlp(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
        string, score, f1score_macro, recall, precision = apply_logistic_regression(X_resampled, X_test, y_resampled, y_test, table[:4] ,db_connection)
        df_ml.loc[len(df_ml)] = [table, string, score, f1score_macro, recall, precision]
        
    return df_ml
        

In [19]:
df_ml = apply_ml_algorithms(connection)

['ADA_1min_complete_1day_preprocessed_1day_features', 'BAT_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BNT_1min_complete_1day_preprocessed_1day_features', 'BSV_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'BTG_1min_complete_1day_preprocessed_1day_features', 'CVC_1min_complete_1day_preprocessed_1day_features', 'DASH_1min_complete_1day_preprocessed_1day_features', 'DCR_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'EOS_1min_complete_1day_preprocessed_1day_features', 'ETC_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'FUN_1min_complete_1day_preprocessed_1day_features', 'HT_1min_complete_1day_preprocessed_1day_features', 'ICX_1min_complete_1day_preprocessed_1day_features', 'IOST_1min_complete_1day_preprocessed_1day_features', 'KNC_1min_complete_1day_preprocessed_1day_f

c:\Users\janfa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\janfa\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\janfa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_star

In [20]:
df_ml

,table_name,model,f1-score weighted,f1-score macro,recall macro,precision macro
0,ADA_1min_complete_1day_preprocessed_1day_features,random forest,0.601006,0.372052,0.370212,0.378777
1,ADA_1min_complete_1day_preprocessed_1day_features,k-nearest neighbour,0.072425,0.097286,0.343434,0.340304
2,ADA_1min_complete_1day_preprocessed_1day_features,support vector classifier,0.379719,0.281650,0.363167,0.342154
3,ADA_1min_complete_1day_preprocessed_1day_features,mlp classifier,0.029379,0.076052,0.042922,0.333333
4,ADA_1min_complete_1day_preprocessed_1day_features,logistic regression,0.442657,0.303335,0.329731,0.371129
...,...,...,...,...,...,...
190,XVG_1min_complete_1day_preprocessed_1day_features,random forest,0.420535,0.281309,0.324062,0.321498
191,XVG_1min_complete_1day_preprocessed_1day_features,k-nearest neighbour,0.377410,0.275235,0.358259,0.362289
192,XVG_1min_complete_1day_preprocessed_1day_features,support vector classifier,0.473826,0.297863,0.322470,0.307784
193,XVG_1min_complete_1day_preprocessed_1day_features,mlp classifier,0.603516,0.279193,0.240183,0.333333
